In [5]:
import requests
import pandas as pd
from datetime import datetime

In [ ]:
# Instructions for creating a new bracket CSV

In [1]:
# 1) make a copy of one of the existing "mm-with-team-codes.csv" files, e.g.:
# cp mm-with-team-codes.csv bracket-data/2025/

In [3]:
# 2) Use the ESPN API to pull the games from march to get a mapping of "official" team name to team code:
year = 2025
data_from_march = requests.get(
    f'https://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard?dates={year}03'
).json()
print(f"Found data for {len(data_from_march['events'])} games!")

short_teams, long_teams = [], []
for event in data_from_march['events']:
    s_teams = event['shortName'].split(' VS ')
    assert len(s_teams) == 2
    l_teams = event['name'].split(' at ')
    assert len(l_teams) == 2
    short_teams += s_teams
    long_teams += l_teams

long_to_short = dict(zip(long_teams, short_teams))

long_to_short

Found data for 64 games!


{'North Carolina Tar Heels': 'UNC',
 'San Diego State Aztecs': 'SDSU',
 'St. Francis (PA) Red Flash': 'SFPA',
 'Alabama State Hornets': 'ALST',
 'Xavier Musketeers': 'XAV',
 'Texas Longhorns': 'TEX',
 "Mount St. Mary's Mountaineers": 'MSM',
 'American University Eagles': 'AMER',
 'SIU Edwardsville Cougars': 'SIUE',
 'Houston Cougars': 'HOU',
 'Auburn Tigers': 'AUB',
 'Wofford Terriers': 'WOF',
 'Tennessee Volunteers': 'TENN',
 'Omaha Mavericks': 'OMA',
 "St. John's Red Storm": 'SJU',
 'Montana Grizzlies': 'MONT',
 'Wisconsin Badgers': 'WIS',
 'UNC Wilmington Seahawks': 'UNCW',
 'Texas Tech Red Raiders': 'TTU',
 'High Point Panthers': 'HPU',
 'Purdue Boilermakers': 'PUR',
 'Yale Bulldogs': 'YALE',
 'Texas A&M Aggies': 'TA&M',
 'McNeese Cowboys': 'MCN',
 'Clemson Tigers': 'CLEM',
 'UC San Diego Tritons': 'UCSD',
 'Michigan Wolverines': 'MICH',
 'VCU Rams': 'VCU',
 'BYU Cougars': 'BYU',
 'Drake Bulldogs': 'DRKE',
 'Missouri Tigers': 'MIZ',
 'Arkansas Razorbacks': 'ARK',
 'Kansas Jayhawks'

In [9]:
# 2) Paste the bracket images and this mapping and ask claude to generate a CSV similar to the existing ones
# using the team names/codes from the mapping above. Optionally tell it who won the "first four" games if they've already happened.
# For example, see thread with Claude: https://claude.ai/share/89dd33e6-ed40-4029-ba99-6d5d6416d583
# NOTE: you do not need the column "odds_api_team_name" yet. We'll add that later
# NOTE: there are some assumptions about order here - consecutive teams play one another,
# and that continues in all rounds, which means you have to have the order of the 4 divisions correct as well.
# Double check that the result looks correct, and paste it into your bracket csv file

In [6]:
from dotenv import dotenv_values

# NOTE: requires paid version of odds api key
config = dotenv_values(".env")

In [8]:
# 3) Pull the "odds api" team names by querying the odds api
# NOTE: UPDATE TO START DATE OF MARCH MADNESS
date_str = datetime(2025, 3, 18, 15, 15).strftime('%Y-%m-%dT%H:%M:%SZ') 
spreads_past = requests.get(
    f'https://api.the-odds-api.com/v4/historical/sports/basketball_ncaab/odds?apiKey={config["ODDS_API_KEY"]}&regions=us&markets=spreads&dateFormat=iso&oddsFormat=american&date={date_str}'
).json()
len(spreads_past['data'])
spreads_past['data'][0].keys()
odds_long_team_names = set()
for game in spreads_past['data']:
    odds_long_team_names.add(game["home_team"])
    odds_long_team_names.add(game["away_team"])

len(odds_long_team_names)

109

In [ ]:
# 4) Prompt claude again to create a mapping between the 64 team names, to resolve any differences in naming
# Example prompt:
f"""
Ok - now I am using a separate API than ESPN to get the spreads for these games. The issue is the official team names in ESPN do not match the official team names required by the odds API. Can you help me construct a mapping from espn long team name to odds api long team name? I have pulled a set of odds API team names that I will provide here (note, this is a superset of the team's that are relevant to the bracket):
{set(odds_long_team_names)}

And as a reminder, here is the set of espn team names:

{set(long_to_short.keys())}

Please create a mapping from espn name to odds api name, and make note of instances where you could not find the corresponding team names. Keep in mind that we only need this map to contain the 64 teams from the bracket so far:

{"PASTE TEAM NAME COLUMN FROM EXISTING CSV"}
"""
# See conversation with claude here: https://claude.ai/share/89dd33e6-ed40-4029-ba99-6d5d6416d583

In [13]:
espn_name_to_odds_api_name = {
    'Auburn Tigers': 'Auburn Tigers',
    'Louisville Cardinals': 'Louisville Cardinals',
    'Creighton Bluejays': 'Creighton Bluejays',
    'Michigan Wolverines': 'Michigan Wolverines',
    'UC San Diego Tritons': 'UC San Diego Tritons',
    'Texas A&M Aggies': 'Texas A&M Aggies',
    'Yale Bulldogs': 'Yale Bulldogs',
    'Ole Miss Rebels': 'Ole Miss Rebels',
    'North Carolina Tar Heels': 'North Carolina Tar Heels',
    'Iowa State Cyclones': 'Iowa State Cyclones',
    'Lipscomb Bisons': 'Lipscomb Bisons',
    'Marquette Golden Eagles': 'Marquette Golden Eagles',
    'New Mexico Lobos': 'New Mexico Lobos',
    'Bryant Bulldogs': 'Bryant Bulldogs',
    'Florida Gators': 'Florida Gators',
    'UConn Huskies': 'UConn Huskies',
    'Oklahoma Sooners': 'Oklahoma Sooners',
    'Memphis Tigers': 'Memphis Tigers',
    'Maryland Terrapins': 'Maryland Terrapins',
    'Missouri Tigers': 'Missouri Tigers',
    'Drake Bulldogs': 'Drake Bulldogs',
    'Texas Tech Red Raiders': 'Texas Tech Red Raiders',
    'UNC Wilmington Seahawks': 'UNC Wilmington Seahawks',
    'Kansas Jayhawks': 'Kansas Jayhawks',
    'Arkansas Razorbacks': 'Arkansas Razorbacks',
    "St. John's Red Storm": "St. John's Red Storm",
    'Omaha Mavericks': 'Omaha Mavericks',
    'Duke Blue Devils': 'Duke Blue Devils',
    'Baylor Bears': 'Baylor Bears',
    'Oregon Ducks': 'Oregon Ducks',
    'Liberty Flames': 'Liberty Flames',
    'Arizona Wildcats': 'Arizona Wildcats',
    'Akron Zips': 'Akron Zips',
    'BYU Cougars': 'BYU Cougars',
    'VCU Rams': 'VCU Rams',
    'Wisconsin Badgers': 'Wisconsin Badgers',
    'Montana Grizzlies': 'Montana Grizzlies',
    "Saint Mary's Gaels": "Saint Mary's Gaels",
    'Vanderbilt Commodores': 'Vanderbilt Commodores',
    'Alabama Crimson Tide': 'Alabama Crimson Tide',
    'Robert Morris Colonials': 'Robert Morris Colonials',
    'Houston Cougars': 'Houston Cougars',
    'Gonzaga Bulldogs': 'Gonzaga Bulldogs',
    'Georgia Bulldogs': 'Georgia Bulldogs',
    'Clemson Tigers': 'Clemson Tigers',
    'McNeese Cowboys': 'McNeese Cowboys',
    'Purdue Boilermakers': 'Purdue Boilermakers',
    'High Point Panthers': 'High Point Panthers',
    'Illinois Fighting Illini': 'Illinois Fighting Illini',
    'Xavier Musketeers': 'Xavier Musketeers',
    'Kentucky Wildcats': 'Kentucky Wildcats',
    'Troy Trojans': 'Troy Trojans',
    'UCLA Bruins': 'UCLA Bruins',
    'Utah State Aggies': 'Utah State Aggies',
    'Tennessee Volunteers': 'Tennessee Volunteers',
    'Wofford Terriers': 'Wofford Terriers',
    # Teams with different naming conventions
    'Alabama State Hornets': 'Alabama St Hornets',
    'Colorado State Rams': 'Colorado St Rams',
    'Grand Canyon Lopes': 'Grand Canyon Antelopes',
    'Michigan State Spartans': 'Michigan St Spartans',
    'Mississippi State Bulldogs': 'Mississippi St Bulldogs',
    "Mount St. Mary's Mountaineers": "Mt. St. Mary's Mountaineers",
    'Norfolk State Spartans': 'Norfolk St Spartans',
    'SIU Edwardsville Cougars': 'SIU-Edwardsville Cougars',
}

In [11]:
# 5) Add a new column for the odds api name:
bracket_df = pd.read_csv('bracket-data/2025/mm-with-team-codes.csv')
bracket_df.head()


,participant_name,team_name,seed,team_code
0,Mike,Auburn Tigers,1,AUB
1,Going,Alabama State Hornets,16,ALST
2,Cimi,Louisville Cardinals,8,LOU
3,Kurta,Creighton Bluejays,9,CREI
4,Nicolai,Michigan Wolverines,5,MICH


In [14]:
bracket_df['odds_api_team_name'] = bracket_df.team_name.apply(lambda n: espn_name_to_odds_api_name[n])
bracket_df.head()

,participant_name,team_name,seed,team_code,odds_api_team_name
0,Mike,Auburn Tigers,1,AUB,Auburn Tigers
1,Going,Alabama State Hornets,16,ALST,Alabama St Hornets
2,Cimi,Louisville Cardinals,8,LOU,Louisville Cardinals
3,Kurta,Creighton Bluejays,9,CREI,Creighton Bluejays
4,Nicolai,Michigan Wolverines,5,MICH,Michigan Wolverines


In [15]:
bracket_df.isna().sum()

participant_name      0
team_name             0
seed                  0
team_code             0
odds_api_team_name    0
dtype: int64

In [17]:
# 6) Save as new file and update your .env with this path!
bracket_df.to_csv('bracket-data/2025/mm-with-team-codes-and-odds-api-name.csv', index=False)

In [ ]:
# 7) Test the bracket construction and pre-populate methods
from bracket import Bracket
config = dotenv_values(".env")  # NOTE: Update with path above!
bracket = Bracket.from_config(config)
print("Original bracket:")
bracket.pretty_print()

In [19]:
# pre-populate to fill games that have already happened
bracket.pre_populate_events()

[2025-04-01 18:27:46,630 - bracket - INFO] Using spread cache!
[2025-04-01 18:27:46,631 - bracket - INFO] No cache file found at: ./.spread_cache.pkl. Creating one!


[2025-04-01 18:27:47,332 - bracket - INFO] Data for 64 events returned.


Found 32 determined events
Got score for this event: round=1 event_id=1 home_participant=Participant(name='Mike', team=Team(name='Auburn Tigers', seed=1, code_name='AUB', odds_api_name='Auburn Tigers', original_position=0), is_in=True) away_participant=Participant(name='Going', team=Team(name='Alabama State Hornets', seed=16, code_name='ALST', odds_api_name='Alabama St Hornets', original_position=1), is_in=True) spread=None spread_final=False status='TBD' team_to_score={} is_complete=False winning_participant=None winning_team_code=None estimated_start_time=None left=None right=None
Found event: round=1 event_id=1 home_participant=Participant(name='Mike', team=Team(name='Auburn Tigers', seed=1, code_name='AUB', odds_api_name='Auburn Tigers', original_position=0), is_in=True) away_participant=Participant(name='Going', team=Team(name='Alabama State Hornets', seed=16, code_name='ALST', odds_api_name='Alabama St Hornets', original_position=1), is_in=True) spread=None spread_final=False sta

[2025-04-01 18:27:48,303 - bracket - INFO] Status update for event from TBD to STATUS_FINAL.
Event #: 1	Auburn Tigers (Mike) vs. Alabama State Hornets (Going)	Score: 83 - 63	Spread: Auburn Tigers -32.0	Status: STATUS_FINAL


Spread: {'Alabama St Hornets': 32.0, 'Auburn Tigers': -32.0}
Got score for this event: round=1 event_id=2 home_participant=Participant(name='Cimi', team=Team(name='Louisville Cardinals', seed=8, code_name='LOU', odds_api_name='Louisville Cardinals', original_position=2), is_in=True) away_participant=Participant(name='Kurta', team=Team(name='Creighton Bluejays', seed=9, code_name='CREI', odds_api_name='Creighton Bluejays', original_position=3), is_in=True) spread=None spread_final=False status='TBD' team_to_score={} is_complete=False winning_participant=None winning_team_code=None estimated_start_time=None left=None right=None
Found event: round=1 event_id=2 home_participant=Participant(name='Cimi', team=Team(name='Louisville Cardinals', seed=8, code_name='LOU', odds_api_name='Louisville Cardinals', original_position=2), is_in=True) away_participant=Participant(name='Kurta', team=Team(name='Creighton Bluejays', seed=9, code_name='CREI', odds_api_name='Creighton Bluejays', original_posit

...

Spread: {'Florida Gators': -7.0, 'Texas Tech Red Raiders': 7.0}
Got score for this event: round=4 event_id=59 home_participant=Participant(name='John (3)', team=Team(name='Duke Blue Devils', seed=1, code_name='DUKE', odds_api_name='Duke Blue Devils', original_position=32), is_in=True) away_participant=Participant(name='Big Mike (3)', team=Team(name='Alabama Crimson Tide', seed=2, code_name='ALA', odds_api_name='Alabama Crimson Tide', original_position=46), is_in=True) spread=None spread_final=False status='TBD' team_to_score={} is_complete=False winning_participant=None winning_team_code=None estimated_start_time=None left=Event(round=3, event_id=53, home_participant=Participant(name='Wasser (3)', team=Team(name='Duke Blue Devils', seed=1, code_name='DUKE', odds_api_name='Duke Blue Devils', original_position=32), is_in=True), away_participant=Participant(name='John (3)', team=Team(name='Arizona Wildcats', seed=4, code_name='ARIZ', odds_api_name='Arizona Wildcats', original_position=38)

[2025-04-01 18:28:37,546 - bracket - INFO] Status update for event from TBD to STATUS_FINAL.
Event #: 59	Duke Blue Devils (John (3)) vs. Alabama Crimson Tide (Big Mike (3))	Score: 85 - 65	Spread: Duke Blue Devils -7.0	Status: STATUS_FINAL


Spread: {'Alabama Crimson Tide': 7.0, 'Duke Blue Devils': -7.0}
Got score for this event: round=4 event_id=60 home_participant=Participant(name='Cimi (4)', team=Team(name='Houston Cougars', seed=1, code_name='HOU', odds_api_name='Houston Cougars', original_position=48), is_in=True) away_participant=Participant(name='Jimmy (4)', team=Team(name='Tennessee Volunteers', seed=2, code_name='TENN', odds_api_name='Tennessee Volunteers', original_position=62), is_in=True) spread=None spread_final=False status='TBD' team_to_score={} is_complete=False winning_participant=None winning_team_code=None estimated_start_time=None left=Event(round=3, event_id=55, home_participant=Participant(name='Kurta (4)', team=Team(name='Houston Cougars', seed=1, code_name='HOU', odds_api_name='Houston Cougars', original_position=48), is_in=True), away_participant=Participant(name='Cimi (4)', team=Team(name='Purdue Boilermakers', seed=4, code_name='PUR', odds_api_name='Purdue Boilermakers', original_position=54), is

[2025-04-01 18:28:38,026 - bracket - INFO] Status update for event from TBD to STATUS_FINAL.
Event #: 60	Houston Cougars (Cimi (4)) vs. Tennessee Volunteers (Jimmy (4))	Score: 69 - 50	Spread: Houston Cougars -3.0	Status: STATUS_FINAL


Spread: {'Houston Cougars': -3.0, 'Tennessee Volunteers': 3.0}


In [21]:
print(f'\nWith events populated:')
bracket.pretty_print()


With events populated:



==================================================Championship==================================================
Event #: 63	Winner of Event # 61 vs. Winner of Event # 62	Score: 0 - 0	Spread: 	Status: TBD



==================================================Semi-Finals==================================================
Event #: 61	Auburn Tigers (Going) vs. Florida Gators (Brey (2))	Score: 0 - 0	Spread: 	Status: TBD
Event #: 62	Duke Blue Devils (John (3)) vs. Houston Cougars (Cimi (4))	Score: 0 - 0	Spread: 	Status: TBD



==================================================Quarter-Finals==================================================
Event #: 57	Auburn Tigers (Going) vs. Michigan State Spartans (Luke)	Score: 70 - 64	Spread: Auburn Tigers -5.5	Status: STATUS_FINAL
Event #: 58	Florida Gators (Brandon (2)) vs. Texas Tech Red Raiders (Brey (2))	Score: 84 - 79	Spread: Florida Gators -7.0	Status: STATUS_FINAL
Event #: 59	Duke Blue Devils (John (3)) vs. Alabama Crims